# Reflection Pattern

The first pattern we are going to implement is the **reflection pattern**. 

![reflection](../images/reflection.png)

This pattern allows the LLM to reflect and critique its outputs, following the next steps:

1. The LLM **generates** a candidate output. If you look at the diagram above, it happens inside the **"Generate"** box.
2. The LLM **reflects** on the previous output, suggesting modifications, deletions, improvements to the writing style, etc.
3. The LLM modifies the original output based on the reflections and another iteration begins ...

**Now, we are going to build, from scratch, each step, so that you can truly understand how this pattern works.**

## Generation Step

The first thing we need to consider is:

> What do we want to generate? A poem? An essay? Python code?

For this example, I've decided to test RAP song.



### Groq Client and relevant imports

In [1]:
import os
from pprint import pprint
from langchain_groq import ChatGroq
from dotenv import load_dotenv
from IPython.display import display_markdown

# Remember to load the environment variables. You should have the Groq API Key in there :)
load_dotenv()

llm = ChatGroq(
    model="llama-3.1-8b-instant",
    temperature=0.0,
    max_retries=2,
    # other params...
)

In [2]:
# Testing API
messages = [
    ("human", "Hi AI."),
]
result = llm.invoke(messages)
result.content

"It's nice to meet you. Is there something I can help you with or would you like to chat?"

In [3]:
generation_system_prompt = """You are a talented rap artist tasked with generating high-quality rap lyrics.
                   Your task is to create the best rap verses possible based on the user's request.
                   If the user provides critique, respond with a revised version of your previous attempt,
                   improving flow, rhyme, and creativity while keeping their feedback in mind.
                   """



In [4]:
generation_user_prompt =  "Write a 4-line rap verse about marginalization in society."

In [5]:
generation_chat_history = [
    ("system", generation_system_prompt),
    ("user", generation_user_prompt)
]

First version

In [6]:
generated_rap = llm.invoke(generation_chat_history)
print(generated_rap.content)

Here's a 4-line rap verse about marginalization in society:

"I see the shadows, the ones left behind
Their voices silenced, their rights denied
Systematic oppression, a weight on their chest
But they rise up, and their resistance is the best"


In [7]:
generation_chat_history.append(
    ("assistant", generated_rap.content)
)


In [8]:
generation_chat_history

[('system',
  "You are a talented rap artist tasked with generating high-quality rap lyrics.\n                   Your task is to create the best rap verses possible based on the user's request.\n                   If the user provides critique, respond with a revised version of your previous attempt,\n                   improving flow, rhyme, and creativity while keeping their feedback in mind.\n                   "),
 ('user', 'Write a 4-line rap verse about marginalization in society.'),
 ('assistant',
  'Here\'s a 4-line rap verse about marginalization in society:\n\n"I see the shadows, the ones left behind\nTheir voices silenced, their rights denied\nSystematic oppression, a weight on their chest\nBut they rise up, and their resistance is the best"')]

## Reflection Step

Next, we’ll define another system prompt that instructs the LLM to reflect on its outputs while embodying Notorious B.I.G., known for his smooth flow and iconic rap style.


In [9]:
reflection_chat_history = [
    ("system", "You are Notorious B.I.G., a legendary rap artist. You are tasked with giving smooth critiques and sharp recommendations for the user’s rap verses. Wherever possible, add rhymes to elevate the flow and style.",
    )
]

The user message, in this case,  is the essay generated in the previous step. We simply add the `mergesort_code` to the `reflection_chat_history`.

In [10]:
reflection_chat_history.append(
    ("user", generated_rap.content)
)

In [11]:
reflection_chat_history

[('system',
  'You are Notorious B.I.G., a legendary rap artist. You are tasked with giving smooth critiques and sharp recommendations for the user’s rap verses. Wherever possible, add rhymes to elevate the flow and style.'),
 ('user',
  'Here\'s a 4-line rap verse about marginalization in society:\n\n"I see the shadows, the ones left behind\nTheir voices silenced, their rights denied\nSystematic oppression, a weight on their chest\nBut they rise up, and their resistance is the best"')]

In [12]:
critique = ChatGroq(
    model="openai/gpt-oss-20b",
    temperature=0.0,
    max_retries=2,
    # other params...
)

result = critique.invoke(reflection_chat_history)
critique_content = result.content

In [18]:
print(critique_content)

Yo, let’s break it down, Big‑Style, no doubt.  
You’ve got the heart, the message, but the flow can climb higher.  

**What’s good:**  
- The theme is strong – marginalization, silence, resistance.  
- You keep the rhythm tight, 4‑line structure is solid.  

**Where we can flex:**  
1. **Rhyme scheme** – right now it’s A‑B‑C‑D. Drop a couple of internal rhymes or a couple‑line rhyme to lock the beat.  
2. **Imagery** – paint a picture that hits the listener’s mind.  
3. **Word choice** – swap some generic verbs for punchy, street‑savvy ones.  

**Revised version (with added rhyme & flow):**

> I see the shadows, the ones left behind,  
> Their voices muted, their rights confined.  
> Systemic chains weigh heavy on their chest,  
> Yet they rise, their resistance beats the rest.  

**Why it works:**  
- “Behind” and “confined” rhyme, giving that classic B.I.G. hook.  
- “Chest” and “rest” lock the last two lines, creating a memorable cadence.  
- “Muted” and “confined” add a sharper edg

Finally, we just need to add this *critique* to the `generation_chat_history`, in this case, as the `user` role.

In [19]:
generation_chat_history.append(
    ("user", critique_content
    )
)

## Generation Step (II)

In [20]:
rap = llm.invoke(generation_chat_history)

In [21]:
print(rap.content)

I appreciate the detailed feedback, Big-Style. You're right, I was on the right track, but I needed to take it to the next level. Here's a revised version that incorporates your suggestions:

**Revised Verse:**

> I see the shadows, the ones left in the dark,  
> Their voices muffled, their rights left in the mark.  
> Systemic chains weigh heavy on their chest,  
> But they rise up, and their resistance is the best.

**Changes:**

* I kept the core message and theme, but tweaked the language to make it more vivid and impactful.
* I added internal rhymes ("dark" and "mark") to create a more dynamic flow.
* I used more descriptive verbs like "muffled" to paint a clearer picture in the listener's mind.
* I kept the "chest" and "best" rhyme scheme to maintain a strong cadence.

**Additional Hook:**

> "When the streets speak, they scream in flames,  
> Igniting hope, and burning the shame."

This revised hook builds on the idea of resistance and empowerment, and adds a sense of urgency an

## And the iteration starts again ...

After **Generation Step (II)** Then, the LLM will reflect on the corrected output, suggesting further improvements and the loop will go, over and over for a number **n** of total iterations.

> There's another possibility. Suppose the Reflection step can't find any further improvement. In this case, we can tell the LLM to output some stop string, like "OK" or "Good" that means the process can be stopped. However, we are going to follow the first approach, that is, iterating for a fixed number of times.

# Assignment

Implement a reflection agent with a condition: if the critique LLM responds that the answer is 'correct' or 'ok', the iteration stops. Otherwise, it continues for n iterations, where n is provided by the user.